In [ ]:
import re
import csv
from functools import lru_cache

def real_sign(i, j):
    """Returns the sign for the priviously quaternion multiplication e_i * e_j. Now the real."""
    table = [
        [1]
    ]
    return table[i][j]

@lru_cache(maxsize=None)
def cayley_dickson_table(n):
    """Generates a multiplication table for the Cayley-Dickson algebra of dimension 2^n."""
    if n == 0:  # Base case: Quaternion table
        dim = 1
        return [[(real_sign(i, j), i ^ j) for j in range(dim)] for i in range(dim)]

    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[None] * dim for _ in range(dim)]

    for i in range(prev_dim):
        for j in range(prev_dim):
            base_val = prev_table[i][j]
            table[i][j] = base_val  # Block 'a'
            new_val = (-base_val[0], base_val[1] + prev_dim)
            table[i][j + prev_dim] = new_val  # Block 'b'
            table[i + prev_dim][j] = new_val  # Block 'c'
            table[i + prev_dim][j + prev_dim] = new_val  # Block 'd'

    for i in range(dim):
        local_row = i % prev_dim
        top = (i < prev_dim)
        for j in range(dim):
            local_col = j % prev_dim
            left = (j < prev_dim)
            s, _ = table[i][j]
            if i == 0 or j == 0:
                s = 1
            if i == j:
                s = 1 if i == 0 else -1
            in_a = top and left
            in_b = top and not left
            in_c = not top and left
            in_d = not top and not left
            if local_row == local_col:
                if in_a or in_b:
                    if n > 0 and local_row == 0:
                        s = real_sign(local_row, local_col)
                    elif local_row == 0:
                        s = 1
                    else:
                        s = -1
                elif in_c:
                    s = 1
                elif in_d:
                    s = -1
            if local_col == 0:
                if in_a or in_b:
                    s = 1
                elif in_c or in_d:
                    s = -1 if local_row == 0 else 1
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    s = 1
                elif in_d:
                    s = -1
            if in_d:
                if local_row == 0 and local_col > 0:
                    s = 1
                if local_col == 0 and local_row > 0:
                    s = -1
            table[i][j] = (s, i ^ j)

    return table

def multiply_elements(a, b, n):
    """Multiplies two hypercomplex elements 'a' and 'b'."""
    table = cayley_dickson_table(n)
    dim = 1 << n
    result = [0] * dim
    for i in range(dim):
        ai = a[i]
        if ai == 0:
            continue
        for j in range(dim):
            bj = b[j]
            if bj == 0:
                continue
            coeff = ai * bj
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """
    Returns a formatted string representation of a hypercomplex element given
    its list of coefficients.
    
    For example, [1, 2, -3, 0] becomes "1 + 2e1 - 3e2".
    """
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        else:
            if coeff == 1:
                term = f"e{i}"
            elif coeff == -1:
                term = f"-e{i}"
            else:
                term = f"{coeff}e{i}"
        terms.append(term)
    if not terms:
        return "0"
    formatted = terms[0]
    for term in terms[1:]:
        formatted += " - " + term[1:] if term.startswith("-") else " + " + term
    return formatted
def parse_element(s, dim):
    """Parses a hypercomplex element string into a list of coefficients."""
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input")
    if s[0] not in "+-":
        s = "+" + s
    tokens = re.findall(r'[+-][^+-]+', s)
    coeffs = [0.0] * dim
    for token in tokens:
        if 'e' in token:
            coeff_str, index_str = token.split('e', 1)
            #coeff = 1.0 if coeff_str in ("+", "-") else float(coeff_str)
            if(coeff_str in ("+")):
                coeff=1.0
            elif (coeff_str in ("-")):
                coeff=-1.0
            else:
                coeff=float(coeff_str)
            index = int(index_str)
        else:
            coeff = float(token)
            index = 0
        if index < 0 or index >= dim:
            raise ValueError(f"Index {index} out of range")
        coeffs[index] += coeff
    return coeffs

def save_table_to_csv(table, filename):
    """Saves the Cayley-Dickson multiplication table to a CSV file."""
    dim = len(table)
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header row (e0, e1, e2, ...)
        header = [f"e{j}" for j in range(dim)]
        writer.writerow([""] + header) # Empty cell in top-left corner

        # Write table data
        for i in range(dim):
            row = [f"e{i}"]  # First element in the row is the basis element
            for sign, index in table[i]:
                row.append(f"{'+' if sign == 1 else '-' }e{index}")
            writer.writerow(row)

def main():
    print("Hypercomplex Element Multiplication & Cayley-Dickson Table Generator")

    try:
        n = int(input("Enter n (n=1 for Complex, n=2 for Quaternions, n=3 for Octonions, etc.): "))
    except ValueError:
        print("Error: n must be an integer.")
        return

    dim = 1 << n
    print(f"\nThis algebra has dimension {dim} (basis elements e0, e1, ..., e{dim-1}).\n")

    # Generate and save the table
    table = cayley_dickson_table(n)
    filename = f"cayley_dickson_table_n{n}.csv"
    save_table_to_csv(table, filename)
    print(f"Cayley-Dickson multiplication table saved to '{filename}'")
    print(f"\nCayley–Dickson Multiplication Table (n={n}, dimension={dim}):")
    for row in table:
        print("\t".join(f"{'' if s == 1 else '-'}e{k}" for s, k in row))
    print("-" * 40)
    # Optional: Perform multiplication (kept from original script)
    while True:
        try:
            a_str = input("Enter the first element (e.g., '1 + 2e1 - 3e2', or 'q' to quit):\n")
            if a_str.lower() == 'q':
                break
            a = parse_element(a_str, dim)

            b_str = input("\nEnter the second element (e.g., '4 - e1 + 0.5e3'):\n")
            b = parse_element(b_str, dim)

            prod = multiply_elements(a, b, n)
            print("\nResult of multiplication:")
            print(f"({format_element(a)}) * ({format_element(b)}) = {format_element(prod)}")

        except ValueError as e:
            print("Error:", e)

if __name__ == "__main__":
    main()

Hypercomplex Element Multiplication & Cayley-Dickson Table Generator

This algebra has dimension 4 (basis elements e0, e1, ..., e3).

Cayley-Dickson multiplication table saved to 'cayley_dickson_table_n2.csv'

Cayley–Dickson Multiplication Table (n=2, dimension=4):
e0	e1	e2	e3
e1	-e0	e3	-e2
e2	-e3	-e0	e1
e3	e2	-e1	-e0
----------------------------------------

Result of multiplication:
(-3.0 + 1.5e1 - e2 + 3.0e3) * (2.0 - 1.5e1 + e2 - 2.0e3) = 3.25 + 6.5e1 - 6.5e2 + 12.0e3
Error: could not convert string to float: '+q'
